In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
matplotlib.rcParams.update({'font.size': 14})

In [5]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [6]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

In [7]:
TRAIN_DATASET_PATH = '../input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '../input/real-estate-price-prediction-moscow/test.csv'

Загрузка данных

**Описание датасета**

* Id - идентификационный номер квартиры
* DistrictId - идентификационный номер района
* Rooms - количество комнат
* Square - площадь
* LifeSquare - жилая площадь
* KitchenSquare - площадь кухни
* Floor - этаж
* HouseFloor - количество этажей в доме
* HouseYear - год постройки дома
* Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности
* Social_1, Social_2, Social_3 - социальные показатели местности
* Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья
* Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров
* Price - цена квартиры

In [8]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
train_df = reduce_mem_usage(train_df)
train_df.tail()

In [9]:
train_df.dtypes

In [10]:
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.tail()

In [11]:
train_df.shape

In [12]:
test_df.shape

Приведение типов

In [13]:
#переводим признаки 'Id' и 'DistrictId' в категориальные
train_df['Id'] = train_df['Id'].astype(str)
train_df['DistrictId'] = train_df['DistrictId'].astype(str) 

# **1. EDA**

**Целевая переменная.** Смотрим целевое значение на графике.

In [14]:
plt.figure(figsize = (16, 8))

train_df['Price'].hist(bins=30)
plt.ylabel('Count')
plt.xlabel('Price')

plt.title('Target distribution')
plt.show()

Видим, что распределение на графике похоже на адекватное: небольшое количество недорогих квартир, много квартир по средней цене, и маленькое количество дорогих квартир. Выбросы не наблюдаем.

Смотрим среднее, медиану и моду целевого значения.

In [15]:
target_mean = round(train_df['Price'].mean(), 2)
target_median = train_df['Price'].median()
target_mode = train_df['Price'].mode()[0]

In [16]:
print(target_mean)

In [17]:
print(target_mode)

Делаем визуализацию полученнных: среднего, медианы и моды.

In [18]:
plt.figure(figsize = (16, 8))

sns.distplot(train_df['Price'], bins=30)

y = np.linspace(0, 0.000007, 10)
plt.plot([target_mean] * 10, y, label='mean', linestyle=':', linewidth=4)
plt.plot([target_median] * 10, y, label='median', linestyle='--', linewidth=4)
plt.plot([target_mode] * 10, y, label='mode', linestyle='-.', linewidth=4)

plt.title('Distribution of price')
plt.legend()
plt.show()

По графику видим, что значения моды и среднего приближены друг к другу, в связи с чем еще раз убеждаемся, что данные достаточно достоверны.

**Количественные переменные.** Смотрим признаки.

In [19]:
train_df.describe()

Видим, что количество объектов у двух признаков ('LifeSquare', 'Healthcare_1') не равно 10000, заявленным в исходных данных, это пропуски, которые необходимо заполнить. Анализируем минимальное и максимальное значения, среднее и медианное значения, видим аномалии в цифрах.

Выбираем количественные признаки.

In [20]:
train_df_num_features = train_df.select_dtypes(include=['float32', 'int8', 'int16', 'int32'])
train_df_num_features.drop('Price', axis=1, inplace=True)

In [21]:
train_df_num_features

In [22]:
train_df_num_features.hist(figsize=(16,16), bins=20, grid=False);

Посмотрим признак 'Square' в разрезе до 200 м2.

In [23]:
train_df.loc[train_df['Square'] < 200, 'Square'].\
    hist(figsize=(4,4), bins=20, grid=False);

Посмотрим признак 'LifeSquare' в разрезе до 200 м2.

In [24]:
train_df.loc[train_df['LifeSquare'] < 200, 'LifeSquare'].\
    hist(figsize=(4,4), bins=20, grid=False);

Посмотрим признак 'KitchenSquare' в разрезе до 50 м2.


In [25]:
train_df.loc[train_df['KitchenSquare'] < 50, 'KitchenSquare'].\
    hist(figsize=(4,4), bins=20, grid=False);

**Номинативные переменные.** Смотрим количество категорий и частоту их встреч в датасете.

In [26]:
train_df.select_dtypes(include='object').columns.tolist()

In [27]:
#сделаем признак'DistrictId' вещественным, перекодируем его по частоте
train_df['DistrictId'].value_counts()

In [28]:
train_df['Ecology_2'].value_counts()

In [29]:
train_df['Ecology_3'].value_counts()

In [30]:
train_df['Shops_2'].value_counts()

**Анализ зависимости таргета от фичей**

In [31]:
grid = sns.jointplot(train_df['Rooms'], train_df['Price'], kind='reg')
grid.fig.set_figwidth(8)
grid.fig.set_figheight(8)

plt.show()

Видим линейную зависимость количества комнат квартиры и цены.

In [32]:
grid = sns.jointplot(train_df['Square'], train_df['Price'], kind='reg')
grid.fig.set_figwidth(8)
grid.fig.set_figheight(8)

plt.show()

Видим линейную зависимость площади квартиры и цены.

In [33]:
grid = sns.jointplot(train_df['Ecology_1'], train_df['Price'], kind='reg')
grid.fig.set_figwidth(8)
grid.fig.set_figheight(8)

plt.show()

Зависимости не наблюдаем.

In [34]:
grid = sns.jointplot(train_df['Social_2'], train_df['Price'], kind='reg')
grid.fig.set_figwidth(8)
grid.fig.set_figheight(8)

plt.show()

Зависимости не наблюдаем.

**Обработка выбросов и пропусков**

In [70]:
class DataPreprocessing:
    """Подготовка исходных данных"""

    def __init__(self):
        """Параметры класса"""
        self.medians = None
        self.kitchen_square_quantile = None
        self.healthcare_median = None
        
    def fit(self, X):
        """Сохранение статистик"""       
        # Расчет медиан
        self.medians = X.median()
        self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
        self.healthcare_median = X['Healthcare_1'].median()
    
    def transform(self, X):
        """Трансформация данных"""

        # Rooms. Заменяем значение признака 'rooms' на 1, 
        #если он был равен 0, и на медианное значение, если он был равен или больше 6.
        X['Rooms_outlier'] = 0
        X.loc[(X['Rooms'] == 0) | (X['Rooms'] >= 6), 'Rooms_outlier'] = 1
        
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        X.loc[X['Rooms'] >= 6, 'Rooms'] = self.medians['Rooms']
        
        # KitchenSquare. Заполняем верхнюю и нижнюю границы адекватных значений в признаке 'KitchenSquare': 
        #кухни больше 13 м2(квантиль 75%) считаем выбросами и заполняем их медианами, 
        #кухни меньше 3 м2 считаем равными 3.
        condition = (X['KitchenSquare'].isna()) \
                    | (X['KitchenSquare'] > self.kitchen_square_quantile)
        
        X.loc[condition, 'KitchenSquare'] = self.medians['KitchenSquare']

        X.loc[X['KitchenSquare'] < 3, 'KitchenSquare'] = 3
        
        # HouseFloor, Floor. Заполняем значения этажности равной 0, и больше 95 этажей медианными значениями.
        #При обнаружении ошибки, где этаж квартиры превосходит общую этажность дома, делаем этаж квартиры 
        #равным этажности дома
        X['HouseFloor_outlier'] = 0
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
        X.loc[X['HouseFloor'] > 95, 'HouseFloor_outlier'] = 1
        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor_outlier'] = 1
        
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']
        X.loc[X['HouseFloor'] > 95, 'HouseFloor'] = self.medians['HouseFloor']
        
        floor_outliers = X.loc[X['Floor'] > X['HouseFloor']].index
        X.loc[floor_outliers, 'Floor'] = X.loc[floor_outliers, 'HouseFloor']
        
        # HouseYear
        current_year = datetime.now().year
        
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
        
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = current_year
        
        # Healthcare_1. Предполагаем, что значение медианы подойдет для заполнения пропусков.
        X['Healthcare_1'] = X['Healthcare_1'].fillna(self.healthcare_median)
            
        # LifeSquare. Рассчитаем и заполним величину 
        #жилой площади квартиры = общая площадь - площадь кухни - 5 м2(санузел и кладовая)
        X['LifeSquare_nan'] = X['LifeSquare'].isna() * 1
        condition = (X['LifeSquare'].isna()) & \
                      (~X['Square'].isna()) & \
                      (~X['KitchenSquare'].isna())
        
        X.loc[condition, 'LifeSquare'] = X.loc[condition, 'Square'] - X.loc[condition, 'KitchenSquare'] - 5
        
        
        X.fillna(self.medians, inplace=True)
        
        return X

In [72]:
preprocessing = DataPreprocessing()
preprocessing.fit(train_df)
train_df = preprocessing.transform(train_df)

In [59]:
train_df.isna().sum()

Пропусков в данных нет.

**Построение новых признаков**

In [ ]:
class FeatureGenetator():
    """Генерация новых фич"""
    
    def __init__(self):
        self.DistrictId_counts = None
        self.binary_to_numbers = None
        self.med_price_by_district = None
        self.med_price_by_floor_year = None
        self.house_year_max = None
        self.floor_max = None
        self.house_year_min = None
        self.floor_min = None
        self.district_size = None
        
    def fit(self, X, y=None):